# NL2SQL (leverages the Azure Accelerator)
#### Import Dependencies
##### (Ctrl/Enter to render Markdown)

In [1]:
# This code block imports necessary libraries and modules for connecting to databases, processing data, 
# and utilizing the LangChain framework for SQL generation from natural language. It also sets up 
# environment variables and defines a function for Markdown display in Jupyter notebooks.
import os
import pandas as pd
import pyodbc
from prompts.prompts import MSSQL_AGENT_PREFIX
from IPython.display import Markdown, HTML, display  
from dotenv import load_dotenv
import logging
import sys
import re

# LangChain dependencies
from langchain_openai import AzureChatOpenAI
from langchain_community.agent_toolkits import create_sql_agent, SQLDatabaseToolkit
from langchain.agents import initialize_agent
from langchain.agents import Tool
from langchain.agents import create_sql_agent  # No longer in agent_toolkits
from langchain.sql_database import SQLDatabase  # Toolkit is replaced with SQLDatabase
from langchain.agents import AgentExecutor

from langchain.callbacks.base import BaseCallbackHandler
from langchain.callbacks.manager import CallbackManager
from langchain.memory import ConversationBufferMemory

from langchain_community.callbacks import get_openai_callback
from langchain_openai import OpenAI

from tiktoken import encoding_for_model
from langchain.agents.conversational_chat.prompt import PREFIX, SUFFIX
from langchain.agents.conversational_chat.base import ConversationalChatAgent
from langchain_core.language_models import BaseLanguageModel
from langchain_core.tools import BaseTool

# Database dependencies
from sqlalchemy import create_engine, inspect, text
from sqlalchemy.engine import URL

# Flag that controls whether to show query execution steps
show_query_execution_steps = True #False #True

# Function to print comments using markdown
def printmd(string):
    display(Markdown(string))

# load secrets from .env file
# Load the .env file from the 'secrets' subfolder
load_dotenv("secrets/credentials.env")

# validate that the environment variables are set
if "AZURE_OPENAI_API_VERSION" not in os.environ:
    raise ValueError("Please set the AZURE_OPENAI_API_VERSION environment variable in the credentials.env file.")
if "AZURE_OPENAI_API_VERSION" not in os.environ:
    raise ValueError("Please set the AZURE_OPENAI_API_VERSION environment variable in the credentials.env file.")
else:
    app_env = os.environ["AZURE_OPENAI_API_VERSION"]

printmd("##### Depedencies loaded...")

# This code block sets up logging for the application. 
# It defines a custom log format that includes the timestamp, 
# logger name, log level, and the log message. # 

# Define a custom log format
log_format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'

# Python Logging Levels:
# DEBUG: Detailed information, typically of interest only when diagnosing problems.
# INFO: Confirmation that things are working as expected.
# WARNING: An indication that something unexpected happened, or indicative of some problem in the near future (e.g., ‘disk space low’). The software is still working as expected.
# ERROR: Due to a more serious problem, the software has not been able to perform some function.
# CRITICAL: A very serious error, indicating that the program itself may be unable to continue running.

# logging.basicConfig(level=logging.DEBUG)
# logging.basicConfig(level=logging.DEBUG, format=log_format)

logging.basicConfig(level=logging.INFO, format=log_format)

logger = logging.getLogger('langchain')

printmd("##### Logger created...")

##### Depedencies loaded...

##### Logger created...

#### Enable Logging

In [2]:
# # This code block sets up logging for the application. 
# # It defines a custom log format that includes the timestamp, 
# # logger name, log level, and the log message. # 

# # Define a custom log format
# log_format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'

# # Python Logging Levels:
# # DEBUG: Detailed information, typically of interest only when diagnosing problems.
# # INFO: Confirmation that things are working as expected.
# # WARNING: An indication that something unexpected happened, or indicative of some problem in the near future (e.g., ‘disk space low’). The software is still working as expected.
# # ERROR: Due to a more serious problem, the software has not been able to perform some function.
# # CRITICAL: A very serious error, indicating that the program itself may be unable to continue running.

# # logging.basicConfig(level=logging.DEBUG)
# # logging.basicConfig(level=logging.DEBUG, format=log_format)

# logging.basicConfig(level=logging.INFO, format=log_format)

# logger = logging.getLogger('langchain')

# printmd("##### Logger created...")

##### Connect to Database and extract metadata -- Using SqlAlchemy and pyodbc   

In [3]:
# Connect to Azure SQL Database using SQLAlchemy and pyodbc. 

#####################################################
## Configure database connection:
#####################################################
db_config = {
    'drivername': 'mssql+pyodbc',
    'username': os.environ["SQL_SERVER_USERNAME"] + '@' + os.environ["SQL_SERVER_NAME"],
    'password': os.environ["SQL_SERVER_PASSWORD"],
    'host': os.environ["SQL_SERVER_NAME"],
    'port': 1433,
    'database': os.environ["SQL_SERVER_DATABASE"],
    'query': {'driver': 'ODBC Driver 17 for SQL Server'},
}

# Create a URL object for connecting to the database
db_url = URL.create(**db_config)

#####################################################
## Test the Database Connection:
#####################################################

# Connect to the Azure SQL Database using the URL string
# engine = create_engine(db_url)

# # inspector = inspect(engine)

# # # Retrieve the list of tables
# # tables = inspector.get_table_names()

# # # Access columns and other schema details
# # for table_name in tables:
# #     columns = inspector.get_columns(table_name)
# #     for column in columns:
# #         print(f"Table: {table_name}, Column: {column['name']}, Type: {column['type']}")
       

# # Test the connection using the SQLAlchemy 2.0 execution style
# with engine.connect() as conn:
#     try:
#         # Use the text() construct for safer SQL execution
#         result = conn.execute(text("SELECT @@VERSION"))
#         version = result.fetchone()
#         print("Connection successful!")
#         print(version)
#     except Exception as e:
#         print(e)
#         sys.exit(1)  # Exit the application with a non-zero status code indicating an error
# ## Test Complete
# #####################################################

# Retrieve and filter table names
# with engine.connect() as conn:
#     inspector = inspect(conn)
#     all_tables = inspector.get_table_names()
#     filtered_tables = [table for table in all_tables if table in ["CheckMaster", "CheckDetail"]]
#     print("Filtered tables:", filtered_tables)

#####################################################
## Create SQLDatabase class
#####################################################
# Connects to and interacts with an Azure SQL Database.
# The class serves as an interface to perform operations like querying data
# and fetching results, integrating with the language model.
#db = SQLDatabase.from_uri(db_url)

try:
    # Create instance of the SQLDatabase class. 
    # Under the hood,SqlAlchemy's create_engine is used to connect to DB's URI.
    # Once connected, SqlAlchemy's MetaData and inspector object are used to intrpspect the DB's schema, 
    # extracting information about tables, columns, and relationships.
    # Using SqlAlchemy's ORM, it can map tables to Python classes and provides methods to query data, 
    # fetch results, and interact with the databae. 
    #SqlAlchemy an ORM. It's phasing out the toolkit, so we will use the SQLDatabase class instead
    db = SQLDatabase.from_uri(db_url)
    #db = SQLDatabase.from_uri(db_url, include_tables=filtered_tables)
except Exception as e:
    print(f"An error occurred while connecting to the database: {e}")
    logger.error(f"An error occurred while connecting to the database: {e}")
    db = None  # Optionally, set db to None or handle the error as needed
    sys.exit(1)  # Exit the application with a non-zero status code indicating an error

# Test the connection to the database by fetching the table names
if show_query_execution_steps:
    print("SqlDatabase Initialized - Read Following Tables: {}".format(db.get_usable_table_names()))

SqlDatabase Initialized - Read Following Tables: ['CheckDetail', 'CheckMaster', 'ClickStreamData', 'HeaderDescriptions', 'SurveyData', 'SurveyMaster', 'clickstream', 'clickstreamfinal', 'sysdiagrams']


##### Initialize AzureOpenAI and Sql Agent

In [4]:
#####################################################
## Initialize AzureChatOpenAI
#####################################################
llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], temperature=0.2, max_tokens=2000, api_version=os.environ["AZURE_OPENAI_API_VERSION"])

# llm = AzureChatOpenAI(
#     azure_deployment="gpt-35-turbo",  # or your deployment
#     api_version="2023-06-01-preview",  # or your api version
#     temperature=0,
#     max_tokens=None,
#     timeout=None,
#     max_retries=2,
    
# printmd(f"LLM Created: {llm.deployment_name}")

system_prompt = """
    You are an SQL expert with deep expertise in generating accurate and efficient SQL queries from natural language.
    Your goal is to understand the user's intent expressed in natural language and translate it into a correct and optimized SQL query that can be executed against the database to retrieve the desired information.
    DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
    If the question does not seem related to the database, just return "I don't know" as the answer.
"""
 
#####################################################
## Create SQLDatabaseToolkit
#####################################################
# SQLDatabaseToolkit is a utility for interacting with a SQL database using a language model (LLM).
# It facilitate the integration between the database and the language model, enabling more sophisticated
# query generation, execution, and result handling.
# Parameters in the SQLDatabase class and the LLM.
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

#####################################################
## Create SqlAgent
#####################################################
# SqlAgent interacts with directly with the SQL database. 
# It leverages the LLM to generate SQL queries from natural language input
# and then execute these queries on the connected database. 
# Note how the SqlAgent leverges the SQLDatabaseToolkit and the language model (LLM). 
try:
    agent_executor = create_sql_agent(
        prefix=MSSQL_AGENT_PREFIX,
        llm=llm,
        #prompt=full_prompt,
        temperature=0.1,
        toolkit=toolkit,
        top_k=30,
        agent_type="openai-tools",
        verbose=show_query_execution_steps, #True,
        agent_executor_kwargs={"return_intermediate_steps": True},
        system_prompt=system_prompt
        # stream_runnble=False
    )
except Exception as e:
    print(f"An error occurred while creating the agent_executor: {str(e)}")
    agent_executor = None  # Set agent_executor to None or handle it as needed
    sys.exit(1)  # Exit the application with a non-zero status code indicating an error

#prompts = agent_executor.get_prompts
# for key in prompts.keys():
#     print(key)

# Continue with the rest of your code, checking if agent_executor was created successfully
if agent_executor:
    printmd(f"##### Agent Executor Created")
else:
    printmd(f"##### Agent Executor Not Created")
    sys.exit(1)  # Exit the application with a non-zero status code indicating an error

##### Agent Executor Created

#### Prompt Engineering

In [5]:
#####################################################
## Set User Prompt
#####################################################
user_prompt = """
		What is the most popular menu item and what are it's total sales?
"""

print("User Prompt: {}".format(user_prompt))

User Prompt: 
		What is the most popular menu item and what are it's total sales?



#### Execute SQL Opeation

In [6]:
#####################################################
## Invoke the Langchain SqlAgent with User Prompt
#####################################################
#
# Experimenal Code to use later:
# final_prompt = f"{PREFIX}\n\n{FORMAT_INSTRUCTIONS.format(tool_names=tool_names)}\n\n{SUFFIX.format(input=input_question, agent_scratchpad=agent_scratchpad)}"
# final_prompt = f"{PREFIX}\n\n{SUFFIX}"

# Invoke the SQL agent with the natural language question
# The agent will generate a SQL query based on the input question   
# and execute the query against the connected database.
# The response contains the query result and other relevant information.
# The response is a dictionary with keys such as 'query', 'result', 'intermediate_steps', and 'execution_time'.
try:
    # Callback() is context manager that provides a callback handler for OpenAI API calls. We use it to track token usage and cost.
    with get_openai_callback() as cb:
        response = agent_executor.invoke(user_prompt)
        # {
        #     "question": QUESTION        
        #     # "chat_history": None
        #     # "callback": cb
        # }
        # response = agent_executor.invoke(QUESTION) 
except Exception as e:
    response = str(e)

#####################################################
## Advanced Logging Block
#####################################################
# This code block iterates through the intermediate steps of the response.
# It extracts the action and observation for each step and prints them.
# The block reveals the agent's decision-making process.
# # 'show_query_execution_steps' enabes/disables this block.
if show_query_execution_steps: 
    # Iterate through each step in the 'intermediate_steps' list from the response
    for step in response['intermediate_steps']:
         # Check if the current step is a dictionary
        if isinstance(step, dict):
            # Extract 'action' and 'observation' from the step, defaulting to empty string if not found
            action = step.get('action', '')
            observation = step.get('observation', '')

        # Check if the current step is a tuple (assumed to be a pair of action and observation)
        elif isinstance(step, tuple):
            # Unpack the tuple directly into action and observation variables
            action, observation = step 

        else:
            # Handle unexpected step types by printing a warning and skipping the current iteration
            print(f"Unexpected step type: {type(step)}")
            continue  # Skip this step if it's neither dict nor tuple

        # Print the action and observation for the current step
        print(f"Action: {action}\nObservation: {observation}\n")



> Entering new SQL Agent Executor chain...


2024-08-27 18:13:52,812 - httpx - INFO - HTTP Request: POST https://ai-west-openai.openai.azure.com//openai/deployments/gpt4-o/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"



Invoking: `sql_db_list_tables` with `{}`


CheckDetail, CheckMaster, ClickStreamData, HeaderDescriptions, SurveyData, SurveyMaster, clickstream, clickstreamfinal, sysdiagrams

2024-08-27 18:13:53,199 - httpx - INFO - HTTP Request: POST https://ai-west-openai.openai.azure.com//openai/deployments/gpt4-o/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"



Invoking: `sql_db_schema` with `{'table_names': 'CheckDetail, CheckMaster'}`



CREATE TABLE [CheckDetail] (
	[CheckKey] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[ActualDate] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[CheckNumber] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[ItemNumber] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Description] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Category] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[ItemSubCategory] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Quantity] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Price] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[EmployeeNumber] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[TableNumber] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[GuestCount] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	CONSTRAINT [FK_CheckDetail_CheckMast

...

```sql
SELECT TOP 1 
    Description, 
    SUM(CAST(Quantity AS DECIMAL(10, 2))) AS TotalQuantity, 
    SUM(CAST(Price AS DECIMAL(10, 2)) * CAST(Quantity AS DECIMAL(10, 2))) AS TotalSales 
FROM 
    CheckDetail 
GROUP BY 
    Description 
ORDER BY 
    TotalQuantity DESC
```

2024-08-27 18:14:08,729 - httpx - INFO - HTTP Request: POST https://ai-west-openai.openai.azure.com//openai/deployments/gpt4-o/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"



Invoking: `sql_db_query` with `{'query': 'SELECT TOP 1 Description, SUM(CAST(Quantity AS DECIMAL(10, 2))) AS TotalQuantity, SUM(CAST(Price AS DECIMAL(10, 2)) * CAST(Quantity AS DECIMAL(10, 2))) AS TotalSales FROM CheckDetail GROUP BY Description ORDER BY TotalQuantity DESC'}`


[('WATER', Decimal('37993.00'), Decimal('0.0000'))]

2024-08-27 18:14:18,915 - httpx - INFO - HTTP Request: POST https://ai-west-openai.openai.azure.com//openai/deployments/gpt4-o/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"


### Final Answer:
The most popular menu item is "WATER" with a total of 37,993 units sold, and its total sales amount to $0.00.

### Explanation:
To determine the most popular menu item and its total sales, I queried the `CheckDetail` table to sum up the quantities and total sales for each item description. I ordered the results by the total quantity in descending order and selected the top result. The SQL query used is:

```sql
SELECT TOP 1 
    Description, 
    SUM(CAST(Quantity AS DECIMAL(10, 2))) AS TotalQuantity, 
    SUM(CAST(Price AS DECIMAL(10, 2)) * CAST(Quantity AS DECIMAL(10, 2))) AS TotalSales 
FROM 
    CheckDetail 
GROUP BY 
    Description 
ORDER BY 
    TotalQuantity DESC
```

This query revealed that "WATER" is the most popular item with 37,993 units sold, and its total sales amount to $0.00.

> Finished chain.
Action: tool='sql_db_list_tables' tool_input={} log='\nInvoking: `sql_db_list_tables` with `{}`\n\n\n' message_log=[AIMessageChunk(content='', additional_kwarg

#### Final Answer

In [7]:
#####################################################
## Craft the response
#####################################################

class SqlExtraction():
    prompt: str
    response: str
    final_answer: str
    output_without_final_answer: str
    sql_statement: str
    prompt_tokens: int
    completion_tokens: int
    total_tokens: int
    total_cost: float

    try:
        print("Total Tokens: {}".format(cb.total_tokens))
        print("Prompt Tokens: {}".format(cb.prompt_tokens))
        print("Completion Tokens: {}".format(cb.completion_tokens))
        print("Total Cost (USD): {}".format(cb.total_cost))
        print()  # Insert a newline

        # Extract the "Final Answer:" part from the response["output"]
        # match = re.search(r'Final Answer:.*?(?=\n\n|$)', response["output"], re.DOTALL)
        # final_answer = match.group(0) if match else "Final Answer not found."

        # Extract the 'Final Answer' from the response["output"]
        match = re.search(r'Final Answer:.*?(?=\n\n|$)', response["output"], re.DOTALL)
        final_answer = match.group(0).strip() if match else "Final Answer not found."

        # Remove the "Final Answer:" part from the response["output"]
        explanation = re.sub(r'Final Answer:.*?(?=\n\n|$)', '', response["output"], flags=re.DOTALL).strip()

        explanation2 = re.search(r'Explanation:.*?(?=\n\n|$)', response["output"], re.DOTALL)

        # sql = re.search(r'SELECT.*', response["output"], re.DOTALL)
        # # Extract the SQL statement from the final answer
        # sql_statement_match = re.search(r'SELECT.*', final_answer, re.DOTALL)
        # sql_statement = sql_statement_match.group(0) if sql_statement_match else "SQL statement not found."

        print("Prompt: {}".format(user_prompt))
        print()  # Insert a newline

        print("Final Answer: {}".format(final_answer))
        print()  # Insert a newline

        # Extract the SQL statement from the response["output"]
        sql_match = re.search(r'SELECT.*?(?=Explanation:|$)', response["output"], re.DOTALL)
        sql_statement = sql_match.group(0).strip() if sql_match else "SQL statement not found."
        print("SQL Statement: {}".format(sql_statement))
        print()  # Insert a newline

        print("Explanation: {}".format(explanation))            
        print()  # Insert a newline
    except Exception as e:
        print(f"An error occurred: {e}")
        raise RuntimeError("An error occurred while extracting the SQL statement.") from e

printmd(f"Complete")

Total Tokens: 727
Prompt Tokens: 521
Completion Tokens: 206
Total Cost (USD): 0.005695

Prompt: 
		What is the most popular menu item and what are it's total sales?


Final Answer: Final Answer:
The most popular menu item is "WATER" with a total of 37,993 units sold, and its total sales amount to $0.00.

SQL Statement: SELECT TOP 1 
    Description, 
    SUM(CAST(Quantity AS DECIMAL(10, 2))) AS TotalQuantity, 
    SUM(CAST(Price AS DECIMAL(10, 2)) * CAST(Quantity AS DECIMAL(10, 2))) AS TotalSales 
FROM 
    CheckDetail 
GROUP BY 
    Description 
ORDER BY 
    TotalQuantity DESC
```

This query revealed that "WATER" is the most popular item with 37,993 units sold, and its total sales amount to $0.00.

Explanation: ### 

### Explanation:
To determine the most popular menu item and its total sales, I queried the `CheckDetail` table to sum up the quantities and total sales for each item description. I ordered the results by the total quantity in descending order and selected the top resul

Complete